<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/PFG_RNN_Predictiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.7 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt
import unicodedata

In [ ]:
# Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
folder_path = '/content/drive/My Drive/PFG_FASPAS/RNN'
docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
print(docs_xlsx)

['RN_LOG.xlsx', 'RN_ACT.xlsx', 'RN_EM.xlsx', 'RN_SO.xlsx', 'RN_AT.xlsx', 'RN_SAAF.xlsx', 'df_8000_rows.xlsx']


In [ ]:
dic_dataframes = {}

for doc in docs_xlsx:
    entire_path = os.path.join(folder_path, doc)
    df = pd.read_excel(entire_path)
    # Uso el nombre del archivo como clave
    dic_dataframes[doc] = df

# Imprimir todas las claves
for clave in dic_dataframes.keys():
    print(clave)

RN_LOG.xlsx
RN_ACT.xlsx
RN_EM.xlsx
RN_SO.xlsx
RN_AT.xlsx
RN_SAAF.xlsx


In [46]:
RN_SAAF = dic_dataframes.get("RN_SAAF.xlsx")
RN_AT = dic_dataframes.get("RN_AT.xlsx")
# RN_ACT = dic_dataframes.get("RN_ACT.xlsx")
RN_EM = dic_dataframes.get("RN_EM.xlsx")
RN_SO = dic_dataframes.get("RN_SO.xlsx")
RN_LOG = dic_dataframes.get("RN_LOG.xlsx")
df_8000rows = dic_dataframes.get("df_8000_rows.xlsx")

In [101]:
# CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
RN_SAAF = pd.read_excel('/content/RN_SAAF.xlsx')
RN_AT = pd.read_excel('/content/RN_AT.xlsx')
# RN_ACT = pd.read_excel('/content/RN_ACT.xlsx')
RN_EM = pd.read_excel('/content/RN_EM.xlsx')
RN_LOG = pd.read_excel('/content/RN_LOG.xlsx')
RN_SO = pd.read_excel('/content/RN_SO.xlsx')
df_8000rows = pd.read_excel('/content/df_8000_rows.xlsx')
provincia_municipio = pd.read_excel('/content/PROVINCIA_MUNICIPIO.xlsx')

In [102]:
# Convert all string columns to uppercase and remove accents/diacritics in combined_df
df_8000rows = df_8000rows.applymap(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)
provincia_municipio = provincia_municipio.applymap(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

In [103]:
# Function to add PROVINCIA column based on LOCALIDAD
def add_provincia_column(df, provincia_mapping):
    df['PROVINCIA'] = df['LOCALIDAD'].map(provincia_mapping)
    return df

# Create a dictionary mapping from LOCALIDAD to PROVINCIA
localidad_to_provincia = provincia_municipio.set_index('MUNICIPIO')['PROVINCIA'].to_dict()

# List of RN dataframes to process
rn_dataframes = {'AT': RN_AT, 'EM': RN_EM, 'LOG': RN_LOG, 'SAAF': RN_SAAF, 'SO': RN_SO}

# Add the PROVINCIA column to each RN dataframe
rn_dfs_with_provincia = {name: add_provincia_column(df, localidad_to_provincia) for name, df in rn_dataframes.items()}

# Drop rows with null values in the 'PROVINCIA' column in each RN dataframe
rn_dfs_cleaned = {name: df.dropna(subset=['PROVINCIA']) for name, df in rn_dfs_with_provincia.items()}

In [104]:
# List of columns to keep
required_columns = ['LOCALIDAD', 'GENERO', 'YEAR NACIMIENTO', 'PROVINCIA']

# Function to ensure the dataframe has all required columns
def ensure_columns(df, columns):
    for column in columns:
        if column not in df.columns:
            df[column] = np.nan  # Create column with NaN values if it does not exist
    return df[columns]  # Return dataframe with only the required columns

# Ensure all dataframes have the required columns
rn_dfs_final = {name: ensure_columns(df, required_columns) for name, df in rn_dfs_cleaned.items()}

<ipython-input-104-605fc2068a84>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = np.nan  # Create column with NaN values if it does not exist


In [105]:
# Mapping dictionary for normalizing gender values
gender_mapping = {
    'FEMENINO': 'MUJER',
    'M': 'MUJER',
    'MASCULINO': 'HOMBRE',
    'H': 'HOMBRE'
}

# Function to normalize the 'GENERO' column in a dataframe
def normalize_gender_column(df):
    df['GENERO'] = df['GENERO'].map(lambda x: gender_mapping.get(x, x))
    return df

# Apply the normalization to the 'SO' dataframe
rn_dfs_final['SO'] = normalize_gender_column(rn_dfs_final['SO'])

<ipython-input-105-333b05a14062>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GENERO'] = df['GENERO'].map(lambda x: gender_mapping.get(x, x))


In [106]:
# Function to insert random registers into a dataframe where the 'GENERO' column is empty
def insert_random_registers(df, gender_distribution):
    # Find the indices where 'GENERO' is empty
    empty_indices = df[df['GENERO'].isna()].index

    # Generate random genders based on the given distribution for the empty indices
    genders = np.random.choice(
        ['MUJER', 'HOMBRE'],
        size=len(empty_indices),
        p=[gender_distribution['MUJER'], gender_distribution['HOMBRE']]
    )

    # Assign these random genders to the empty 'GENERO' rows
    df.loc[empty_indices, 'GENERO'] = genders
    return df

# Example of usage
gender_distribution = {'MUJER': 0.44, 'HOMBRE': 0.56}

# Insert random registers into a specific dataframe, e.g., 'ACT', where 'GENERO' is empty
rn_dfs_final['SAAF'] = insert_random_registers(rn_dfs_final['SAAF'], gender_distribution)
rn_dfs_final['SO'] = insert_random_registers(rn_dfs_final['SO'], gender_distribution)

In [107]:
# Get the min and max year in the 'YEAR NACIMIENTO' column
min_year = int(rn_dfs_final['SO']['YEAR NACIMIENTO'].min())
max_year = int(rn_dfs_final['SO']['YEAR NACIMIENTO'].max())

# Function to generate random years with the specified distribution
def generate_random_years(size):
    # 75% of the years under 2004
    under_2004 = np.random.randint(min_year, 2004, size=int(size * 0.75))
    # 25% of the years between 2004 and max_year
    over_2004 = np.random.randint(2004, max_year + 1, size=size - int(size * 0.75))
    return np.concatenate([under_2004, over_2004])

# Function to insert random years into the 'YEAR NACIMIENTO' column where it is empty
def insert_random_years(df):
    # Find the indices where 'YEAR NACIMIENTO' is empty
    empty_indices = df[df['YEAR NACIMIENTO'].isna()].index

    # Generate random years based on the specified distribution
    random_years = generate_random_years(len(empty_indices))
    np.random.shuffle(random_years)  # Shuffle to mix the values

    # Assign these random years to the empty 'YEAR NACIMIENTO' rows
    df.loc[empty_indices, 'YEAR NACIMIENTO'] = random_years
    return df

# Apply the function to the 'SO' dataframe
rn_dfs_final['SO'] = insert_random_years(rn_dfs_final['SO'])

In [108]:
# Add a column named 'SERVICIO' with the name of the dataframe to each dataframe in rn_dfs_final
def add_servicio_column(dfs):
    for name, df in dfs.items():
        df['SERVICIO'] = name
    return dfs

# Apply the function to rn_dfs_final
rn_dfs_final = add_servicio_column(rn_dfs_final)

<ipython-input-108-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SERVICIO'] = name
<ipython-input-108-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SERVICIO'] = name
<ipython-input-108-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [109]:
# Combine all dataframes into a single dataframe
combined_df = pd.concat(rn_dfs_final.values(), ignore_index=True)

# Shuffle the rows of the combined dataframe
shuffled_combined_df = combined_df.sample(frac=1).reset_index(drop=True)

shuffled_combined_df['SERVICIO'].value_counts()

SERVICIO
SO      1457
SAAF     567
EM       203
LOG      182
AT       143
Name: count, dtype: int64

In [110]:
# Crear una función para categorizar por grupos de edad
def categorize_age(year):
    if year < 1940:
        return 'ANTES DE 1940'
    elif year < 1960:
        return '1940-1959'
    elif year < 1980:
        return '1960-1979'
    elif year < 2000:
        return '1980-1999'
    elif year < 2020:
        return '2000-2020'
    else:
        return '2020 EN ADELANTE'

rna1 = shuffled_combined_df.copy()
# Aplicar la función a la columna 'YEAR NACIMIENTO'
rna1['YEAR NACIMIENTO'] = rna1['YEAR NACIMIENTO'].apply(categorize_age)

# Ver el resultado
print(rna1)

                 LOCALIDAD  GENERO YEAR NACIMIENTO    PROVINCIA SERVICIO
0              GUADALAJARA  HOMBRE       1960-1979  GUADALAJARA     SAAF
1     CABANILLAS DEL CAMPO   MUJER       2000-2020  GUADALAJARA      LOG
2                 ALBACETE  HOMBRE       2000-2020     ALBACETE       AT
3            VILLARROBLEDO  HOMBRE       1980-1999     ALBACETE     SAAF
4              GUADALAJARA  HOMBRE       2000-2020  GUADALAJARA       SO
...                    ...     ...             ...          ...      ...
2547                TOLEDO  HOMBRE       1980-1999       TOLEDO       EM
2548              FONTANAR  HOMBRE       1980-1999  GUADALAJARA       SO
2549            CHILOECHES  HOMBRE       2000-2020  GUADALAJARA       SO
2550   AZUQUECA DE HENARES  HOMBRE       1960-1979  GUADALAJARA       SO
2551           GUADALAJARA  HOMBRE       1980-1999  GUADALAJARA       SO

[2552 rows x 5 columns]


##RNA predictiva con los datos reales
Vamos a realizar una primera red neuronal con los registros reales.

####Preparación de los datos

Convertimos las variables categóricas en variables dummy usando pd.get_dummies.

Separamos las características (x) de la etiqueta (y).
Convertimos la etiqueta en variables dummy para la clasificación multiclase.

In [111]:
# Convertir las variables categóricas en variables dummy
df = pd.get_dummies(rna1, columns=['LOCALIDAD', 'GENERO', 'PROVINCIA', 'YEAR NACIMIENTO'])

# Separar las características (X) y la etiqueta (y)
X = df.drop('SERVICIO', axis=1)
y = df['SERVICIO']

# Convertir la etiqueta en variables dummy
y = pd.get_dummies(y)

####División de los datos
Usaremos un 70% de los datos para entrenar y del 30% restante, un 70% para validar (21% del total) e ir ajustandio los hiperparámetros (evitando el sobreentrenamiento) y un 30% para el conjunto de test (9% del total), para evaluar el modelo y su precisión al final.

In [112]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

# Verificar las formas de los conjuntos de datos
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (1786, 228)
Tamaño del conjunto de validación: (536, 228)
Tamaño del conjunto de prueba: (230, 228)


In [113]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Definir el modelo
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50
56/56 [==============================] - 2s 13ms/step - loss: 1.3502 - accuracy: 0.5302 - val_loss: 1.1546 - val_accuracy: 0.5466
Epoch 2/50
56/56 [==============================] - 0s 6ms/step - loss: 1.1302 - accuracy: 0.5817 - val_loss: 1.0817 - val_accuracy: 0.5466
Epoch 3/50
56/56 [==============================] - 0s 7ms/step - loss: 1.0765 - accuracy: 0.5845 - val_loss: 1.0385 - val_accuracy: 0.5634
Epoch 4/50
56/56 [==============================] - 0s 5ms/step - loss: 1.0341 - accuracy: 0.6030 - val_loss: 0.9953 - val_accuracy: 0.6101
Epoch 5/50
56/56 [==============================] - 0s 4ms/step - loss: 1.0024 - accuracy: 0.6271 - val_loss: 0.9602 - val_accuracy: 0.6306
Epoch 6/50
56/56 [==============================] - 0s 5ms/step - loss: 0.9636 - accuracy: 0.6249 - val_loss: 0.9117 - val_accuracy: 0.6381
Epoch 7/50
56/56 [==============================] - 0s 5ms/step - loss: 0.9366 - accuracy: 0.6361 - val_loss: 0.8885 - val_accuracy: 0.6474
Epoch 8/50
56/56 [=

In [114]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import relu, tanh

# Definir el modelo con hiperparámetros ajustables
def create_model(learning_rate=0.001, activation='relu', dropout_rate=0.5):
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation=activation))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_model(learning_rate=0.0005, activation='tanh', dropout_rate=0.3)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100
28/28 [==============================] - 2s 13ms/step - loss: 1.3716 - accuracy: 0.4905 - val_loss: 1.1661 - val_accuracy: 0.5466
Epoch 2/100
28/28 [==============================] - 0s 6ms/step - loss: 1.0999 - accuracy: 0.5997 - val_loss: 1.0381 - val_accuracy: 0.5989
Epoch 3/100
28/28 [==============================] - 0s 5ms/step - loss: 1.0213 - accuracy: 0.6333 - val_loss: 0.9892 - val_accuracy: 0.6175
Epoch 4/100
28/28 [==============================] - 0s 5ms/step - loss: 0.9744 - accuracy: 0.6461 - val_loss: 0.9482 - val_accuracy: 0.6343
Epoch 5/100
28/28 [==============================] - 0s 5ms/step - loss: 0.9534 - accuracy: 0.6484 - val_loss: 0.9280 - val_accuracy: 0.6269
Epoch 6/100
28/28 [==============================] - 0s 5ms/step - loss: 0.9337 - accuracy: 0.6579 - val_loss: 0.9088 - val_accuracy: 0.6437
Epoch 7/100
28/28 [==============================] - 0s 7ms/step - loss: 0.9249 - accuracy: 0.6557 - val_loss: 0.8948 - val_accuracy: 0.6549
Epoch 8/100


In [115]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Definir el modelo con hiperparámetros ajustables
def create_simple_model(learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_simple_model(learning_rate=0.001, dropout_rate=0.3)

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50
56/56 [==============================] - 3s 16ms/step - loss: 1.4739 - accuracy: 0.3936 - val_loss: 1.2517 - val_accuracy: 0.5466
Epoch 2/50
56/56 [==============================] - 1s 10ms/step - loss: 1.1601 - accuracy: 0.5801 - val_loss: 1.0745 - val_accuracy: 0.5466
Epoch 3/50
56/56 [==============================] - 1s 9ms/step - loss: 1.0672 - accuracy: 0.5857 - val_loss: 1.0208 - val_accuracy: 0.5560
Epoch 4/50
56/56 [==============================] - 1s 9ms/step - loss: 1.0213 - accuracy: 0.6075 - val_loss: 0.9851 - val_accuracy: 0.6138
Epoch 5/50
56/56 [==============================] - 1s 9ms/step - loss: 0.9836 - accuracy: 0.6305 - val_loss: 0.9465 - val_accuracy: 0.6213
Epoch 6/50
56/56 [==============================] - 0s 5ms/step - loss: 0.9752 - accuracy: 0.6321 - val_loss: 0.9194 - val_accuracy: 0.6250
Epoch 7/50
56/56 [==============================] - 0s 5ms/step - loss: 0.9427 - accuracy: 0.6377 - val_loss: 0.8926 - val_accuracy: 0.6474
Epoch 8/50
56/56 [

##RNA predictiva con datos generados
Red Neuronal con 8000 registros más que han sido creados teniendo en cuenta las estadísticas reales de los datos, es decir, contemplando cómo se distribuyen por edad, género y localidad y, obviamente, el municipio y la provincia correctamente computados.

Esto se hace para intentar aumentar el porcentaje de precisión obtenido.

In [116]:
# Calculate the frequency of each element in the 'SERVICIO' column
service_frequency = df['SERVICIO'].value_counts()

# Group by 'PROVINCIA' and 'SERVICIO' and count the occurrences
service_province_freq = shuffled_combined_df.groupby(['PROVINCIA', 'SERVICIO']).size().reset_index(name='COUNT')

# Function to assign services based on frequency
def assign_service(row, freq_df):
    province = row['PROVINCIA']
    possible_services = freq_df[freq_df['PROVINCIA'] == province]
    if possible_services.empty:
        return np.nan
    else:
        return np.random.choice(possible_services['SERVICIO'].values, p=possible_services['COUNT'].values / possible_services['COUNT'].sum())

# Apply the function to assign services
df_8000rows['SERVICIO'] = df_8000rows.apply(assign_service, axis=1, args=(service_province_freq,))

# Ensure balanced classes by sampling
min_class_size = service_frequency.min()
balanced_df = df_8000rows.groupby('SERVICIO').apply(lambda x: x.sample(n=min_class_size, replace=True)).reset_index(drop=True)

In [117]:
# Calculate the frequency of each service
service_frequency = shuffled_combined_df['SERVICIO'].value_counts()

# Set the target number of records per service
target_size = 1457

# Create a balanced dataframe starting with the initial shuffled data
to_use_df = shuffled_combined_df.copy()

# Loop through each service to balance the dataframe
for service, count in service_frequency.items():
    if count < target_size:
        # Get the rows for the current service
        service_df = balanced_df[balanced_df['SERVICIO'] == service]
        # Sample additional rows with replacement to reach the target size
        additional_samples = service_df.sample(n=target_size - count, replace=True)
        to_use_df = pd.concat([to_use_df, additional_samples])

# Shuffle the final balanced dataframe
to_use_df = to_use_df.sample(frac=1).reset_index(drop=True)

In [118]:
# Shuffle the rows of the dataframe ready to use
to_use_df = to_use_df.sample(frac=1).reset_index(drop=True)

In [119]:
# Get the min and max year in the 'YEAR NACIMIENTO' column
min_year = int(to_use_df['YEAR NACIMIENTO'].min())
max_year = int(to_use_df['YEAR NACIMIENTO'].max())

# Function to generate random years with the specified distribution
def generate_random_years(size):
    return np.random.randint(min_year, max_year + 1, size=int(size))

# Function to insert random years into the 'YEAR NACIMIENTO' column where it is empty
def insert_random_years(df):
    # Find the indices where 'YEAR NACIMIENTO' is empty
    empty_indices = df[df['YEAR NACIMIENTO'].isna()].index

    # Generate random years based on the specified distribution
    random_years = generate_random_years(len(empty_indices))

    # Assign these random years to the empty 'YEAR NACIMIENTO' rows
    df.loc[empty_indices, 'YEAR NACIMIENTO'] = random_years
    return df

# Apply the function to the dataframe
to_use_df = insert_random_years(to_use_df)

In [120]:
# Crear una función para categorizar por grupos de edad
def categorize_age(year):
    if year < 1940:
        return 'Antes de 1940'
    elif year < 1960:
        return '1940-1959'
    elif year < 1980:
        return '1960-1979'
    elif year < 2000:
        return '1980-1999'
    else:
        return '2000 en adelante'

# Aplicar la función a la columna 'YEAR NACIMIENTO'
to_use_df['YEAR NACIMIENTO'] = to_use_df['YEAR NACIMIENTO'].apply(categorize_age)

# Ver el resultado
print(to_use_df.head())

   LOCALIDAD  GENERO   YEAR NACIMIENTO PROVINCIA SERVICIO
0  MOTILLEJA  HOMBRE         1980-1999  ALBACETE       EM
1     YUNCOS   MUJER  2000 en adelante    TOLEDO       SO
2   ALBACETE  HOMBRE  2000 en adelante  ALBACETE       SO
3    ALCARAZ   MUJER  2000 en adelante  ALBACETE       SO
4  CANAVERAS   MUJER         1940-1959    CUENCA       EM


In [121]:
# Determina la cantidad de servicios únicos
unique_services = to_use_df['SERVICIO'].unique()
num_services = len(unique_services)

# Determina cuántos registros por servicio necesitas para el conjunto de prueba
test_size_per_service = 85 // num_services

# Si hay más de 85 registros, ajusta la muestra para que esté balanceada
if test_size_per_service * num_services < 85:
    additional_samples = 85 - (test_size_per_service * num_services)
else:
    additional_samples = 0

# Crear el conjunto de prueba balanceado
test_df = pd.DataFrame()

for service in unique_services:
    service_df = to_use_df[to_use_df['SERVICIO'] == service]
    test_samples = service_df.sample(n=test_size_per_service, replace=False)
    test_df = pd.concat([test_df, test_samples])

# Añadir muestras adicionales si es necesario
if additional_samples > 0:
    additional_df = to_use_df[~to_use_df.index.isin(test_df.index)]
    additional_samples_df = additional_df.sample(n=additional_samples, replace=False)
    test_df = pd.concat([test_df, additional_samples_df])

# Eliminar las filas seleccionadas del dataframe original para crear el conjunto de entrenamiento/validación
to_use_df = to_use_df.drop(test_df.index)

# Verifica el conjunto de prueba
print(test_df['SERVICIO'].value_counts())
print(test_df.shape)

# Verifica el conjunto de entrenamiento/validación
print(to_use_df.shape)

SERVICIO
EM      17
SO      17
AT      17
LOG     17
SAAF    17
Name: count, dtype: int64
(85, 5)
(7200, 5)


#Red Neuronal Predictiva

Creación de una red neuronal predictiva que sea capaz de clasificar nuevos usuarios con buena precisión en servicio y provincia.

In [126]:
# Convertir las variables categóricas en variables dummy
df = pd.get_dummies(to_use_df, columns=['LOCALIDAD', 'GENERO', 'PROVINCIA', 'YEAR NACIMIENTO'])

# Separar las características (X) y la etiqueta (y)
X = df.drop('SERVICIO', axis=1)
y = df['SERVICIO']

# Convertir la etiqueta en variables dummy
y = pd.get_dummies(y)

In [127]:
# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

# Verificar las formas de los conjuntos de datos
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (5040, 502)
Tamaño del conjunto de validación: (1512, 502)
Tamaño del conjunto de prueba: (648, 502)


In [128]:
# Definir el modelo
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50
158/158 [==============================] - 2s 7ms/step - loss: 1.5579 - accuracy: 0.2909 - val_loss: 1.4021 - val_accuracy: 0.4266
Epoch 2/50
158/158 [==============================] - 1s 6ms/step - loss: 1.2929 - accuracy: 0.4744 - val_loss: 1.0287 - val_accuracy: 0.6124
Epoch 3/50
158/158 [==============================] - 1s 5ms/step - loss: 0.9969 - accuracy: 0.6274 - val_loss: 0.7503 - val_accuracy: 0.7176
Epoch 4/50
158/158 [==============================] - 1s 5ms/step - loss: 0.7776 - accuracy: 0.7234 - val_loss: 0.6244 - val_accuracy: 0.7738
Epoch 5/50
158/158 [==============================] - 1s 5ms/step - loss: 0.6543 - accuracy: 0.7653 - val_loss: 0.5607 - val_accuracy: 0.7996
Epoch 6/50
158/158 [==============================] - 1s 4ms/step - loss: 0.5861 - accuracy: 0.7897 - val_loss: 0.5194 - val_accuracy: 0.8148
Epoch 7/50
158/158 [==============================] - 1s 5ms/step - loss: 0.5455 - accuracy: 0.8085 - val_loss: 0.4906 - val_accuracy: 0.8241
Epoch 

In [129]:
# Definir el modelo con hiperparámetros ajustables
def create_model(learning_rate=0.001, activation='relu', dropout_rate=0.5):
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation=activation))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_model(learning_rate=0.0005, activation='tanh', dropout_rate=0.3)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100
79/79 [==============================] - 2s 10ms/step - loss: 1.4717 - accuracy: 0.3734 - val_loss: 1.3186 - val_accuracy: 0.4716
Epoch 2/100
79/79 [==============================] - 0s 6ms/step - loss: 1.2121 - accuracy: 0.5238 - val_loss: 1.0779 - val_accuracy: 0.6091
Epoch 3/100
79/79 [==============================] - 0s 5ms/step - loss: 0.9690 - accuracy: 0.6581 - val_loss: 0.8472 - val_accuracy: 0.7262
Epoch 4/100
79/79 [==============================] - 0s 6ms/step - loss: 0.7760 - accuracy: 0.7272 - val_loss: 0.7273 - val_accuracy: 0.7315
Epoch 5/100
79/79 [==============================] - 0s 5ms/step - loss: 0.6831 - accuracy: 0.7516 - val_loss: 0.6816 - val_accuracy: 0.7401
Epoch 6/100
79/79 [==============================] - 0s 6ms/step - loss: 0.6406 - accuracy: 0.7554 - val_loss: 0.6641 - val_accuracy: 0.7460
Epoch 7/100
79/79 [==============================] - 0s 6ms/step - loss: 0.6142 - accuracy: 0.7536 - val_loss: 0.6424 - val_accuracy: 0.7526
Epoch 8/100


In [130]:
# Definir el modelo con hiperparámetros ajustables
def create_simple_model(learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_simple_model(learning_rate=0.001, dropout_rate=0.3)

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50
158/158 [==============================] - 3s 7ms/step - loss: 1.5477 - accuracy: 0.2792 - val_loss: 1.3960 - val_accuracy: 0.4888
Epoch 2/50
158/158 [==============================] - 1s 5ms/step - loss: 1.2572 - accuracy: 0.4974 - val_loss: 1.0011 - val_accuracy: 0.6667
Epoch 3/50
158/158 [==============================] - 1s 3ms/step - loss: 0.9356 - accuracy: 0.6641 - val_loss: 0.7482 - val_accuracy: 0.7586
Epoch 4/50
158/158 [==============================] - 1s 3ms/step - loss: 0.7423 - accuracy: 0.7458 - val_loss: 0.6248 - val_accuracy: 0.7765
Epoch 5/50
158/158 [==============================] - 0s 3ms/step - loss: 0.6274 - accuracy: 0.7843 - val_loss: 0.5618 - val_accuracy: 0.8036
Epoch 6/50
158/158 [==============================] - 1s 4ms/step - loss: 0.5721 - accuracy: 0.7974 - val_loss: 0.5464 - val_accuracy: 0.8089
Epoch 7/50
158/158 [==============================] - 1s 4ms/step - loss: 0.5323 - accuracy: 0.8054 - val_loss: 0.5121 - val_accuracy: 0.8228
Epoch 